#  SQL-Python Assignment

In [310]:
import psycopg2
from datetime import datetime
from datetime import date

* First let's take the current date. This will be our reference when taking the daily snapshot.

In [296]:
curr_date = '2014-08-14'
ts = datetime.strptime(curr_date, '%Y-%m-%d').strftime("%Y%m%d")
print(ts)

20140814


In [297]:
# Before running the following command it's necessary to create a
# database using postgres shell. Then load the socialmedia.sql 
# file to the social media database.
# After this is done, we can run the next command

conn = psycopg2.connect(dbname='socialmedia', host='localhost', user = 'postgres', password = 'feliz1')

* Create table with the necessary data for the assignment

In [298]:
c = conn.cursor()

c.execute(
    ''' CREATE TABLE logins_7d(
    userid integer,
    reg_date timestamp without time zone,
    last_login timestamp without time zone,
    logins_7d integer,
    logins_7d_mobile integer,
    logins_7d_web integer,
    opt_out varchar(3)
    );''')

conn.commit()
conn.close()

* In the following section the tables and querys for the assignment will be made.

In [304]:
conn = psycopg2.connect(dbname='socialmedia', host='localhost', user = 'postgres', password = 'feliz1')

In [305]:
c = conn.cursor()

In [306]:
c.execute(
    ''' CREATE TEMP TABLE temp AS 
    SELECT userid, SUM(CASE WHEN logins.type = 'web' THEN 1 ELSE 0 END) as cnt_web, SUM(CASE WHEN logins.type = 'mobile' THEN 1 ELSE 0 END) AS cnt_mobile
    FROM logins
    WHERE logins.tmstmp > timestamp %(ts)s - interval '7 days'
    GROUP BY userid
    ORDER BY userid;
    ''',{'ts':ts}
)

conn.commit()

c.execute(
    ''' CREATE TEMP TABLE temp2 AS
    SELECT userid, MAX(tmstmp) AS last_log
    FROM logins
    WHERE logins.tmstmp > timestamp %(ts)s - interval '7 days'
    GROUP BY userid
    ORDER BY userid;
    ''',{'ts':ts}
    )

conn.commit()

c.execute(
    '''CREATE TEMP TABLE temp3 AS
    SELECT userid, COUNT(*) AS cnt
    FROM logins
    WHERE logins.tmstmp > timestamp %(ts)s - interval '7 days'
    GROUP BY userid
    ORDER BY userid;
''',{'ts':ts})

conn.commit()

c.execute(
    '''CREATE TEMP TABLE opt_out AS
    SELECT userid, COUNT(*) AS opt_cnt
    FROM optout
    GROUP BY userid ORDER BY userid;

''')

conn.commit()

In [307]:
c.execute(
    '''INSERT INTO logins_7d 
        SELECT temp.userid AS userid,registrations.tmstmp AS reg_date, temp2.last_log AS last_login, 
        temp3.cnt AS logins_7d, temp.cnt_mobile AS logins_7d_mobile, temp.cnt_web AS logins_7d_web, opt_out.opt_cnt AS optout
        FROM temp
        LEFT JOIN temp2 ON temp.userid = temp2.userid
        LEFT JOIN temp3 ON temp.userid = temp3.userid
        LEFT JOIN opt_out ON temp.userid = opt_out.userid
        JOIN registrations ON temp.userid = registrations.userid;
    
    ''')

conn.commit()


In [308]:
query = ('''
    SELECT * FROM temp LIMIT 10;
''')

query2 = ('''
    SELECT * FROM temp2 LIMIT 10;
''')

query3 = ('''
    SELECT * FROM temp3 LIMIT 10;
''')

query4 = (
    ''' SELECT * FROM logins_7d LIMIT 20;
    ''')
c.execute(query4)
c.fetchmany(15)

[(0,
  datetime.datetime(2014, 6, 23, 22, 25, 59),
  datetime.datetime(2014, 8, 13, 13, 5, 23),
  3,
  3,
  0,
  None),
 (1,
  datetime.datetime(2013, 12, 21, 20, 11, 2),
  datetime.datetime(2014, 8, 12, 20, 49, 22),
  6,
  2,
  4,
  None),
 (2,
  datetime.datetime(2014, 4, 18, 10, 5, 11),
  datetime.datetime(2014, 8, 14, 8, 20),
  5,
  4,
  1,
  None),
 (3,
  datetime.datetime(2013, 12, 17, 16, 1, 59),
  datetime.datetime(2014, 8, 13, 19, 59, 38),
  4,
  3,
  1,
  None),
 (4,
  datetime.datetime(2014, 8, 11, 19, 22, 32),
  datetime.datetime(2014, 8, 9, 5, 10, 7),
  1,
  1,
  0,
  None),
 (5,
  datetime.datetime(2013, 8, 31, 5, 22, 58),
  datetime.datetime(2014, 8, 10, 2, 5, 54),
  1,
  1,
  0,
  None),
 (6,
  datetime.datetime(2013, 8, 18, 10, 9, 28),
  datetime.datetime(2014, 8, 12, 18, 44, 34),
  1,
  0,
  1,
  None),
 (7,
  datetime.datetime(2014, 3, 21, 22, 55, 47),
  datetime.datetime(2014, 8, 12, 5, 4, 12),
  2,
  2,
  0,
  None),
 (8,
  datetime.datetime(2014, 5, 3, 1, 48, 29),

In [309]:
conn.close()